# Working interface models

In [ ]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np
from ibvpy.api import TStepBC , TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface
from ibvpy.fets import FETS2D4Q
from ibvpy.fets.fets1D5 import FETS1D52ULRH
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from bmcs_matmod.slide.vslide_34_TN_axisym import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic
from bmcs_matmod.slide.slide_explorer import SlideExplorer

# Single material point

In [ ]:
%%capture
import numpy as np
material_params = dict(
    E_T=1e6, gamma_T=0, K_T=0, S_T=100, c_T=1, bartau=6, 
    E_N=1e6, S_N=300, c_N = 1, m = 0.1, f_t=6, f_c=500, f_c0=400, eta=0.)

eta_range = [0]
se_N_list = []
se_T_list = []

for eta in eta_range:
    material_params['eta'] = eta
    se1 = SlideExplorer(n_steps=300, k_max=50)
    se1.slide_model.trait_set(**material_params)
    se1.trait_set(s_x_1 = 0, s_y_1 = 0, w_1 = 0.0001);
    se1.run()
    se_N_list.append(se1)



    se2 = SlideExplorer(n_steps=300, k_max=50)
    se2.slide_model.trait_set(**material_params)
    se2.trait_set(s_x_1 =  0.0001, s_y_1 = 0, w_1 = 0);
    se2.run()
    se_T_list.append(se2)

In [ ]:
import matplotlib.pylab as plt
fig, (ax_N,  ax_T) = plt.subplots(1,2, tight_layout=True, figsize=(10,4))
fig.canvas.header_visible=False

for se1, se2 in zip(se_N_list, se_T_list):

    tau_x_pi_1, tau_y_pi_1, sig_pi_1, Z_1, X_x_1, X_y_1, Y_T_1, Y_N_1 = se1.Sig_arr.T
    tau_x_pi_2, tau_y_pi_2, sig_pi_2, Z_2, X_x_2, X_y_2, Y_T_2, Y_N_2 = se2.Sig_arr.T

    w_1 = np.linspace(0, se1.w_1, n_step_1)
    s_x_2 = np.linspace(0, se2.s_x_1, n_step_2)

    ax_N.plot(w_1, sig_pi_1)
                 
    ax_N.set_xlabel('$w$')
    ax_N.set_ylabel('$\sigma_N$')

    ax_T.plot(s_x_2, tau_x_pi_2)
    ax_T.set_xlabel('$s_x$')
    ax_T.set_ylabel('$\sigma_T$')

# FE Axisymmetric discretization

In [ ]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 1.0
P = 1.0
R_in = P / (2 * np.pi)
# R_in = 1
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                         coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                         coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=37000, nu=0.18)
m2 = MATS3DElastic(E=37000, nu=0.18)

xd12 = XDomainFEInterface(
    integ_factor =  2 * np.pi * R_in,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [ ]:
material_params =  dict(
     E_T=1e7, gamma_T=0, K_T=0, S_T=5, c_T=6, bartau=3, 
     E_N=1e7, S_N=5, c_N = 3, m = 0.1, f_t=2, f_c=60, f_c0 = 40, eta=0.)
bond_m = Slide34(**material_params)

In [ ]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ]
)

## Compression/Traction fc

In [ ]:
lower_fixed_1 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
lower_fixed_0 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)

lower_slide_dummy = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=0.001)
lower_slide_dofs = lower_slide_dummy.dofs


upper_compression_slice = BCSlice(slice=xd_upper.mesh[:, -1, :, -1],
         var='u', dims=[1], value=0.01)

compression_dofs = upper_compression_slice.dofs
compression = -20 / len(compression_dofs)
upper_compression_force_first = [BCDof(var='f', dof=dof, value = compression) 
             for dof in compression_dofs ]
bc1 = [lower_fixed_0, lower_fixed_1] + upper_compression_force_first

In [ ]:
compression

In [ ]:
m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 100
s.tline.step = 0.01
s.tstep.fe_domain.serialized_subdomains

In [ ]:
xd12.hidden = True
s.reset()
s.run()

In [ ]:
F_to = m.hist.F_t
U_to = m.hist.U_t
U_compression = np.average(U_to[:, compression_dofs], axis=-1)
U_slide = np.average(U_to[:, lower_slide_dofs], axis=-1)

states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [ ]:
time = m.hist.t
f, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(10, 3))

ax1.plot(U_compression, sig_pi_N[:,0,0].flatten(), label='sigma')
ax1.plot(U_compression, sig_pi_N[:,0,1].flatten(), label='sigma')
ax1.set_xlabel('displacement [mm]')
ax1.set_ylabel('normal stress')
ax1.legend()

ax2.plot(U_slide, sig_pi_Tx[:,0,0].flatten(), color='red', label='tau')
ax2.set_xlabel('displacement [mm]')
ax2.set_ylabel('tangential stress')
ax2.legend()

ax3.plot(time, omega_Nx[:,0,0].flatten(), color='red', label='normal damage')
ax3.plot(time, omega_Tx[:,0,0].flatten(), color='green', label='tangential damage')

ax3.set_xlabel('time')
ax3.set_ylabel('damage')
ax3.legend()

ax4.plot(time, u_pi_N[:,0,0].flatten(), color='red', label='opening')
ax4.set_xlabel('time')
ax4.set_ylabel('opening')
ax4.legend()
plt.tight_layout()


In [ ]:
# if True:
#     mlab.options.backend = 'envisage'
#     mlab.options.offscreen = False # 'envisage'
#     f_strain = mlab.figure()
#     scene = mlab.get_engine().scenes[-1]
#     scene.name = 'stress'
#     strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
#     strain_viz.setup()
#     strain_viz.warp_vector.filter.scale_factor = 0.1
#     strain_viz.plot(s.tstep.t_n)
#     mlab.show()

## Sliding dc

In [ ]:
material_params =  dict(
     E_T=1e7, gamma_T=1e5, K_T=0, S_T=50000, c_T=6, bartau=3, 
     E_N=1e7, S_N=50000, c_N = 3, m = 0.05, f_t=2, f_c=60, f_c0 = 40, eta=0.)
bond_m = Slide34(**material_params)

In [ ]:
lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)

lower_slide = BCSlice(slice=xd_lower.mesh[-1, :, -1, :], var='u', dims=[0], value=0.0002)
lower_slide_dofs = lower_slide_dummy.dofs

upper_compression_slice = BCSlice(slice=xd_upper.mesh[:, -1, :, -1],
         var='u', dims=[1], value=0.01)

compression_dofs = upper_compression_slice.dofs

bc1 =   [lower_fixed_0, upper_fixed_1, lower_slide]

In [ ]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ]
)

In [ ]:
m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 100
s.tline.step = 0.01
s.tstep.fe_domain.serialized_subdomains

In [ ]:
xd12.hidden = True
s.reset()
s.run()

In [ ]:
F_to = m.hist.F_t
U_to = m.hist.U_t
U_compression = np.average(U_to[:, compression_dofs], axis=-1)
U_slide = np.average(U_to[:, lower_slide_dofs], axis=-1)

states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [ ]:
time = m.hist.t
f, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(10, 3))

ax1.plot(U_compression, sig_pi_N[:,0,0].flatten(), label='sigma')
ax1.plot(U_compression, sig_pi_N[:,0,1].flatten(), label='sigma')
ax1.set_xlabel('displacement [mm]')
ax1.set_ylabel('normal stress')
ax1.legend()

ax2.plot(U_slide, sig_pi_Tx[:,0,0].flatten(), color='red', label='tau')
ax2.set_xlabel('displacement [mm]')
ax2.set_ylabel('tangential stress')
ax2.legend()

ax3.plot(time, omega_Nx[:,0,0].flatten(), color='red', label='normal damage')
ax3.plot(time, omega_Tx[:,0,0].flatten(), color='green', label='tangential damage')

ax3.set_xlabel('time')
ax3.set_ylabel('damage')
ax3.legend()

ax4.plot(time, u_pi_N[:,0,0].flatten(), color='red', label='opening')
ax4.set_xlabel('time')
ax4.set_ylabel('opening')
ax4.legend()
plt.tight_layout()


In [ ]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

## Sliding fc

In [ ]:
lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
upper_fixed_1 = BCSlice(slice=xd_upper.mesh[-1, :, 0-1, :], var='u', dims=[0], value=0)

lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=0)
Force = 4 / len(lower_slide.dofs)
lower_slide_force = [BCDof(var='f', dof=dof, value = Force) 
            for dof in lower_slide.dofs]

bc1 =   [lower_fixed_0, upper_fixed_1] + lower_slide_force

In [ ]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ]
)

In [ ]:
m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 100
s.tline.step = 0.01
s.tstep.fe_domain.serialized_subdomains

In [ ]:
xd12.hidden = True
s.reset()
s.run()

In [ ]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [ ]:
time = m.hist.t
f, (ax1, ax2, ax3, ax4) = plt.subplots(1,4)
ax1.plot(time, sig_pi_N[:,0,0].flatten(), label='sigma')
ax1.plot(time, sig_pi_N[:,0,1].flatten(), label='sigma')
ax1.set_xlabel('time')
ax1.set_ylabel('normal stress')
ax1.legend()

ax2.plot(time, sig_pi_Tx[:,0,0].flatten(), color='red', label='tau')
ax2.set_xlabel('time')
ax2.set_ylabel('tangential stress')
ax2.legend()

ax3.plot(time, omega_Nx[:,0,0].flatten(), color='red', label='normal damage')
ax3.plot(time, omega_Tx[:,0,0].flatten(), color='green', label='tangential damage')

ax3.set_xlabel('time')
ax3.set_ylabel('damage')
ax3.legend()

ax4.plot(time, u_pi_N[:,0,0].flatten(), color='red', label='opening')
ax4.set_xlabel('time')
ax4.set_ylabel('opening')
ax4.legend()

In [ ]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()